<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Predicción de aterrizaje de la primera etapa del Falcon 9 de SpaceX**

## Recopilación de registros de lanzamientos de Falcon 9 y Falcon Heavy desde Wikipedia


Tiempo estimado necesario: **40** minutos

En este laboratorio, realizarás un raspado web para recopilar registros históricos de lanzamientos de Falcon 9 de una página de Wikipedia titulada "List of Falcon 9 and Falcon Heavy launches".

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


El primer escenario del Falcon 9 aterrizará con éxito.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Aquí se muestran varios ejemplos de aterrizajes fallidos:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


Más específicamente, los registros de lanzamiento se almacenan en una tabla HTML que se muestra a continuación:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectivos

Web scraping de registros de lanzamiento del Falcon 9 con `BeautifulSoup`:
- Extraer una tabla HTML de registros de lanzamiento del Falcon 9 de Wikipedia.
- Analizar la tabla y convertirla en un DataFrame de Pandas.

Primero importemos los paquetes necesarios para este laboratorio.


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [27]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

y proporcionaremos algunas funciones auxiliares para procesar la tabla HTML extraída de la web.


In [28]:
def date_time(table_cells):
    """
    Esta función devuelve la fecha y la hora de la celda de la tabla HTML.
    Entrada: el elemento de una celda de datos de tabla extrae una fila adicional.
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    Esta función devuelve la versión del propulsor desde la celda de la tabla HTML.
    Entrada: el elemento de una celda de datos de tabla extrae una fila adicional.
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    Esta función devuelve el estado de aterrizaje desde la celda de la tabla HTML.
    Entrada: el elemento de una celda de datos de tabla extrae una fila adicional.
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    Esta función devuelve el estado de aterrizaje desde la celda de la tabla HTML.
    Entrada: el elemento de una celda de datos de tabla extrae una fila adicional.
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filtrar los nombres de dígitos y vacíos
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name


Para mantener consistentes las tareas del laboratorio, se te pedirá que obtengas los datos de una captura de la página de Wikipedia "List of Falcon 9 and Falcon Heavy launches" actualizada el `9 de junio de 2021`.

In [29]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

A continuación, solicita la página HTML desde la URL proporcionada y obtén un objeto de `respuesta` (response).

### TAREA 1: Solicitar la página Wiki de lanzamientos de Falcon 9 desde su URL

Primero, realicemos un método HTTP GET para solicitar la página HTML de lanzamientos de Falcon 9 como una respuesta HTTP.


In [30]:
# Utiliza el método requests.get() con el static_url proporcionado
# Asigna la respuesta a un objeto
response = requests.get(static_url)


Crea un objeto `BeautifulSoup` a partir de la respuesta HTML.


In [31]:
# Usa BeautifulSoup() para crear un objeto BeautifulSoup a partir del contenido de texto de la respuesta.

soup = BeautifulSoup(response.text, 'html.parser')


Imprime el título de la página para verificar si el objeto `BeautifulSoup` se creó correctamente. 



In [32]:
# Usa el atributo `soup.title`.
print(soup.title)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>



### TAREA 2: Extraer todos los nombres de columnas/variables del encabezado de la tabla HTML

A continuación, queremos recopilar todos los nombres de columnas relevantes del encabezado de la tabla HTML.


Intentemos encontrar todas las tablas en la página wiki primero. Si necesitas refrescar la memoria sobre `BeautifulSoup`, por favor revisa el enlace de referencia externa al final de este laboratorio.

In [33]:
# Utiliza la función `find_all` en el objeto BeautifulSoup,
# con el tipo de elemento `table`. Asigna el resultado a una lista llamada `html_tables`.
html_tables = soup.find_all('table')


Empezando desde la tercera tabla, nuestra tabla objetivo contiene los registros reales de lanzamientos.


In [34]:
# Imprimiré la tercera tabla y verificaré su contenido.
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

Deberías poder ver los nombres de las columnas incrustados en los elementos de encabezado de la tabla `<th>` de la siguiente manera:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


A continuación, simplemente necesitamos iterar a través de los elementos `<th>` y aplicar la función `extract_column_from_header()` proporcionada para extraer los nombres de las columnas uno por uno.


In [35]:

column_names = []

# Aplicar la función find_all() con el elemento `th` en first_launch_table
# Iterar sobre cada elemento th y aplicar la función extract_column_from_header() para obtener un nombre de columna
# Agregar el nombre de columna no vacío (`if name is not None and len(name) > 0`) a una lista llamada column_names

for header in first_launch_table.find_all('th'):
    name = extract_column_from_header(header)
    if name is not None and len(name) > 0:
        column_names.append(name)


Revisa los nombres de columna extraídos.

In [36]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TAREA 3: Crea un dataframe analizando las tablas HTML de lanzamientos


Crearemos un diccionario vacío con claves de los nombres de columna extraídos en la tarea anterior. Más adelante, este diccionario se convertirá en un dataframe de Pandas.


In [37]:
launch_dict = dict.fromkeys(column_names)

# Eliminar una columna no relevante
del launch_dict['Date and time ( )']

# Inicialicemos launch_dict con cada valor como una lista vacía
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Se agregaron algunas columnas nuevas
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []


Luego, simplemente necesitamos llenar el `launch_dict` con los registros de lanzamiento extraídos de las filas de la tabla.


Por lo general, las tablas HTML en páginas Wiki suelen contener anotaciones inesperadas y otros tipos de ruido, como enlaces de referencia `B0004.1[8]`, valores faltantes `N/A [e]`, formato inconsistente, etc.


Para simplificar el proceso de análisis, hemos proporcionado un fragmento de código incompleto a continuación para ayudarte a completar el `launch_dict`. Por favor, completa el siguiente fragmento de código con los comentarios de TODO o puedes optar por escribir tu propia lógica para analizar todas las tablas de lanzamiento:

In [38]:
extracted_row = 0
# Extraer cada tabla 
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Obtener fila de la tabla 
    for rows in table.find_all("tr"):
        # Revisar si el primer encabezado de la tabla es un número correspondiente a un número de lanzamiento 
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        # Obtener elementos de la tabla 
        row = rows.find_all('td')
        # Si es un número, guardar las celdas en un diccionario 
        if flag:
            extracted_row += 1
            # Valor del número de vuelo
            # TODO: Agregar flight_number a launch_dict con la clave `Flight No.`
            # print(flight_number)
            
            datatimelist = date_time(row[0])
            
            # Valor de la fecha
            # TODO: Agregar la fecha a launch_dict con la clave `Date`
            date = datatimelist[0].strip(',')
            # print(date)
            
            # Valor de la hora
            # TODO: Agregar la hora a launch_dict con la clave `Time`
            time = datatimelist[1]
            # print(time)
              
            # Versión del propulsor
            # TODO: Agregar bv a launch_dict con la clave `Version Booster`
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string
            print(bv)
            
            # Sitio de lanzamiento
            # TODO: Agregar el sitio de lanzamiento a launch_dict con la clave `Launch Site`
            launch_site = row[2].a.string
            # print(launch_site)
            
            # Carga útil
            # TODO: Agregar la carga útil a launch_dict con la clave `Payload`
            payload = row[3].a.string
            # print(payload)
            
            # Masa de la carga útil
            # TODO: Agregar la masa de la carga útil a launch_dict con la clave `Payload mass`
            payload_mass = get_mass(row[4])
            # print(payload)
            
            # Órbita
            # TODO: Agregar la órbita a launch_dict con la clave `Orbit`
            orbit = row[5].a.string
            # print(orbit)
            
            # Cliente
            # TODO: Agregar el cliente a launch_dict con la clave `Customer`
            customer = row[6].a.string
            # print(customer)
            
            # Resultado del lanzamiento
            # TODO: Agregar el resultado del lanzamiento a launch_dict con la clave `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            # print(launch_outcome)
            
            # Aterrizaje del propulsor
            # TODO: Agregar el aterrizaje del propulsor a launch_dict con la clave `Booster landing`
            booster_landing = landing_status(row[8])
            # print(booster_landing)
            
            


F9 v1.0B0003.1
F9 v1.0B0004.1
F9 v1.0B0005.1
F9 v1.0B0006.1
F9 v1.0B0007.1
F9 v1.1B1003
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 FT
F9 v1.1
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT♺
F9 FT
F9 FT
F9 FT
F9 FTB1029.2
F9 FT
F9 FT
F9 B4
F9 FT
F9 B4
F9 B4
F9 FTB1031.2
F9 B4
F9 FTB1035.2
F9 FTB1036.2
F9 B4
F9 FTB1032.2
F9 FTB1038.2
F9 B4
F9 B4B1041.2
F9 B4B1039.2
F9 B4
F9 B5B1046.1
F9 B4B1043.2
F9 B4B1040.2
F9 B4B1045.2
F9 B5
F9 B5B1048
F9 B5B1046.2
F9 B5
F9 B5B1048.2
F9 B5B1047.2
F9 B5B1046.3
F9 B5
F9 B5
F9 B5B1049.2
F9 B5B1048.3
F9 B5[268]
F9 B5
F9 B5B1049.3
F9 B5B1051.2
F9 B5B1056.2
F9 B5B1047.3
F9 B5
F9 B5
F9 B5B1056.3
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5B1058.2
F9 B5
F9 B5B1049.6
F9 B5
F9 B5B1060.2
F9 B5B1058.3
F9 B5B1051.6
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5
F9 B5B1051.8
F9 B5B1058.5


AttributeError: 'NoneType' object has no attribute 'string'

Después de que hayas llenado los valores de los registros de lanzamiento analizados en `launch_dict`, puedes crear un marco de datos (dataframe) a partir de él.


In [39]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [40]:
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time


 Podemos exportarlo a un archivo <b>CSV</b> para la siguiente sección, pero para mantener las respuestas consistentes y en caso de que tengas dificultades para completar este laboratorio.

Los laboratorios siguientes utilizarán un conjunto de datos proporcionado para hacer que cada laboratorio sea independiente.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [41]:
df.to_csv('spacex_web_scraped.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
